In [32]:
# 数据分析/处理
import numpy as np
import pandas as pd
import re

# 搭建神经网络
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchtext
from torch import optim
from torch.utils.data import Dataset,DataLoader

# 数据可视化
import matplotlib.pyplot as plt
import warnings

# word2vec
from gensim.models import Word2Vec


warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# 宇宙的答案
np.random.seed(42)
torch.manual_seed(42)

### 关于词嵌入（Word Embedding）

词嵌入（Word Embedding）指的是根据词汇在文本中的上下文将词汇转化为稠密化的向量（也称为分布式表示）表示的一类算法。之所以称这种技术为嵌入（Embedding）是由于它可以将词汇表示在一个向量空间中，我们甚至可以使用这些表示（一般简称为词向量）进行一些带有语义的运算（例如：king-man+woman=queen，或计算余弦相似度）。最早的基于DL的词嵌入技术Word2Vec由传奇捷克NLP研究员Tomas Mikolov于2013年前后与Ilya、Jeff Dean等人于Google共同开发。

Word2Vec的基本假设就是：每个词汇的含义，取决于它**可能**出现的上下文（事后证明这个假设基本上是正确的）。Word2Vec突破了之前的各种词汇表示方法无法联系上下文的缺点,可以显示词之间的相似关系，且稠密的向量化的表示更适于计算和存储。但是Word2Vec仍然有一些缺点，例如由于反义词由于出现的语境比较接近，所以两个反义词在向量空间中比较接近，正常来讲一对反义词的词向量应该成一个平角才对。

GloVe则是斯坦福大学提出的针对Word2Vec的改进，相较于Word2Vec改进了面对生僻词时的等情况下的性能（尽管Mikolov仍然认为GloVe的效果逊于Word2Vec）。

根据原项目的要求，应当使用以下三种方式对词汇进行表示：
 + Word2Vec（对应原要求中的word embedding，这样处理是因为Word2Vec影响力太大，所以有时有人会用word embedding指代它）
 + 随机初始化
 + GloVe

然后搭建RNN和CNN进行分类。

欲了解更多可以参考以下资料：

[Word2Vec的介绍1](https://zhuanlan.zhihu.com/p/61635013)

[Word2Vec的介绍2](https://zhuanlan.zhihu.com/p/26306795)

[GloVe的介绍1](https://zhuanlan.zhihu.com/p/50946044)

[GloVe的介绍2](https://zhuanlan.zhihu.com/p/42073620)

[GloVe的介绍3](https://zh.gluon.ai/chapter_natural-language-processing/glove.html)

另外，Word2Vec和GloVe并不是唯一的一种分布式词向量表示方法。另一类常用的分布式词表示方法还有ELMo和Bert，这类方法可以通过考察词语所在的上下文语境动态的形成词向量。

### 载入数据\处理数据

对数据的分析同task1 

Word2Vec使用Gensim库内置的实现，在我们自己的语料库中进行训练

随机初始化使用torch原生的随机化Embedding

GloVe使用斯坦福原生的预训练权重


In [8]:
# 数据在task1文件架中
test=pd.read_csv("../task1/test.tsv", delimiter="\t").drop(columns=["PhraseId","SentenceId"])
train=pd.read_csv("../task1/train.tsv", delimiter="\t").drop(columns=["PhraseId","SentenceId"])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Phrase     156060 non-null  object
 1   Sentiment  156060 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [16]:
maxSenLen=0
for i in train["Phrase"].to_list():
    maxSenLen=max(maxSenLen,len(i))
print(maxSenLen)

283


In [9]:
# from task1.ipynb
from nltk.corpus import stopwords

def tokenization_to_ngram(sentence,n=1):
    '''
    将句子转化为token,去除停止词,并返回用于n-gram语言建模的特征
    '''
    stop_words = set(stopwords.words('english'))
    # print(type(sentence))
    words = re.sub("[^\w]", " ",  sentence).split() 
    filtered_sentence=[w.lower() for w in words if not w in stop_words]
    output=[]
    if(n!=1):
        if(len(filtered_sentence)<n):
            # 对于n>len(filtered_sentence)的情况，直接将句子中所有的词拼接
            tmp=''
            for i in range(len(filtered_sentence)):
                tmp=tmp+filtered_sentence[i]
            output.append(tmp)
            return output
        else:
            # 对于其他情况，将句子分解为n个词一份
            for i in range(len(filtered_sentence)-n+1):
                # 这一步是将n个单词拼在一起作为一个单词，这样的话可以视作一个单词，方便一会儿进行哈希
                tmp=filtered_sentence[i]
                for t in range(1,n):
                    tmp+=filtered_sentence[i+t]
                output.append(tmp)
            return output
    else:
        return filtered_sentence


In [13]:
# 将句子转化为列表
sentence_list=pd.concat([train["Phrase"],test["Phrase"]],axis=0,ignore_index=True)\
                .apply(tokenization_to_ngram,n=1).to_list()
sentence_list[0]

['a',
 'series',
 'escapades',
 'demonstrating',
 'adage',
 'good',
 'goose',
 'also',
 'good',
 'gander',
 'occasionally',
 'amuses',
 'none',
 'amounts',
 'much',
 'story']

### 数据集类

### 建模  
使用CNN和RNN进行分类，使用Dropout防止过拟合

### RNN

参考：[pytorch官方文档](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)

In [31]:

class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,out_size,num_layers=1,bid=False,dropout=0.1) -> None:
        super(RNN,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.num_layers=num_layers
        self.bid=bid
        self.rnn=nn.RNN(input_size,hidden_size,
                        num_layers,batch_first=True,dropout=dropout,bidirectional=bid)
        self.dropout=nn.Dropout(dropout)
        self.project=nn.Linear(hidden_size,out_size)
    
    def forward(self,X):
        # deft: self.bid*self.num_layers=1, batch_size=X.size(0) 
        h0=torch.zeros(self.bid*self.num_layers,X.size(0),self.hidden_size)
        X,_=self.rnn(X,h0)
        X=self.dropout(X)
        output=self.project(X)
        return output 

### CNN

In [ ]:
class TextCNN(nn.Module):
    def __init__(self) -> None:
        super(TextCNN).__init__()

    def forward(self,X):
        return 

### Word2Vec初始化

In [24]:
# 词向量维度为50维,sg=1表示使用skip-gram
model1=Word2Vec(sentence_list,vector_size=50,sg=1)
model1.save("Word2Vec.task2.model")

In [30]:
def sen2word2vec(sentence_list,model,maxlen=maxSenLen):
    vecList=model.wv[sentence_list]
    l,_=vecList.shape
    if(l==maxlen):
        return vecList
    else:
        for i in range(maxSenLen-l):
            z=np.zeros((1,50))
            vecList=np.append(vecList,z,axis=0)
    return vecList
s=['a','none','good']
p=sen2word2vec(s,model1)
print(p.shape)

(283, 50)


### 随机向量初始化

In [ ]:
class RandomEmbedding(nn.Module):
    def __init__(self,vocab_size,d_model) -> None:
        super(RandomEmbedding,self).__init__()
        self.embedding=nn.Embedding()
    def forward(self,X):
        return self.embedding(X)

### GloVe初始化

In [6]:
del train
del test